In [1]:
#imports and user defined errors 
import re
import copy

class GateNotDefined(Exception):
    '''
    Error to be raised when the gate is not found in the the basic gates 
    '''
    def __init__(self, gate):
        self.message = f"{gate} not defined"
        super().__init__(self.message)

class GateReprError(Exception):
    '''
    Error to be raised for illegal gate declaration format 
    '''
    def __init__(self, gate_repr):
        self.message = f"{gate_repr} not valid"
        super().__init__(self.message)

#custom errors to be used
class FloatingOutput(Exception):
    '''
    Error to be raised when an output of a gate is floating
    '''
    def __init__(self, node_name):
        self.message = f"{node_name} is floating, no driver for output node is defined"
        super().__init__(self.message)

class CirNotLevelized(Exception):
    '''
    Error to be raised if a function that requires levelized circuit is called
    before the circuit is levelized
    '''
    def __init__(self):
        self.message = f"Circuit is not levelized! Levelize circuit and try again"
        super().__init__(self.message)

class InputUndefined(Exception):
    '''
    This error is raised when a circuit is tried to simulated without input assignment
    '''
    def __init__(self, node):
        self.message = f"{node.name} not assigned an input value"
        super().__init__(self.message)

class FaultBeforeFaultsite(Exception):
    '''
    This error is raised when a fault is detected before the fault site
    '''
    def __init__(self, fault, output):
        self.message = f"Fault value of {output} observed before fault site {fault}"
        super().__init__(self.message)

class FaultReprError(Exception):
    '''
    This error is raised when the fault input from the user is not of the expected format
    '''
    def __init__(self, fault_input = None, fault_output = None, fault_type = None):
        self.message = "Fault representaiton Error!"
        if fault_input != None:
            self.message = self.message + f"{fault_input} not a node in the circuit"
        elif fault_output != None:
            self.message = self.message + f"{fault_output} not a node in the circuit"
        elif fault_type != None:
            self.message = self.message + f"sa{fault_type} not a valid fault"


In [2]:
#basic classes used for circuit representation
class node_type:
    input_node = 0
    output_node = 1
    internal_wire = 3

#class to represnt the value of each node
class node_value:
    zero = 0
    one = 1
    d = 2
    d_bar = 3
    undefined = -1
    str_repr = {zero: "0", one: "1", d: "D", d_bar:"D'", undefined: "X"}

class gate_type:
    AND_gate = 0
    OR_gate = 1
    NOT_gate = 2
    NAND_gate = 3
    NOR_gate = 4
    XOR_gate = 5
    XNOR_gate = 6
    BUFF_gate = 7

    @staticmethod
    def AND(input_list, output_node):
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.zero or output == node_value.undefined:
                return output
            if input_list[i].get_value(output_node) == node_value.zero:
                output =  node_value.zero
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output = node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output =  node_value.zero
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output =  node_value.zero
                else:
                    output = node_value.d_bar

        return output
    
    @staticmethod
    def OR(input_list, output_node):
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.one or output == node_value.undefined:
                return output
            if input_list[i].get_value(output_node) == node_value.one:
                output =  node_value.one
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output =  node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output = node_value.one
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output = node_value.one
                else:
                    output = node_value.d_bar

        return output
    
    @staticmethod
    def NOT(input_list, output_node):
        output_dict = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        return output_dict[input_list[0].get_value(output_node)]
    
    @staticmethod
    def BUFF(input_list, output_node):
        
        return input_list[0].get_value(output_node)
    
    @staticmethod
    def NAND(input_list, output_node):
        inv_output = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.zero or output == node_value.undefined:
                return inv_output[output]
            if input_list[i].get_value(output_node) == node_value.zero:
                output =  node_value.zero
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output = node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output =  node_value.zero
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output =  node_value.zero
                else:
                    output = node_value.d_bar

        return inv_output[output]
    
    @staticmethod
    def NOR(input_list, output_node):
        inv_output = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.one or output == node_value.undefined:
                return inv_output[output]
            if input_list[i].get_value(output_node) == node_value.one:
                output =  node_value.one
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output =  node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output = node_value.one
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output = node_value.one
                else:
                    output = node_value.d_bar

        return inv_output[output]
    
    @staticmethod
    def XOR(input_list, output_node):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = None
        if input_list[0].get_value(output_node) == node_value.undefined:
            return node_value.undefined
        elif input_list[0].get_value(output_node) == node_value.zero:
            output = input_list[1].get_value(output_node)
        elif input_list[0].get_value(output_node) == node_value.one:
            output = inverted_output[input_list[1].get_value(output_node)]
        elif input_list[0].get_value(output_node) == node_value.d:
            if input_list[1].get_value(output_node) == node_value.d_bar:
                output = node_value.one
            elif input_list[1].get_value(output_node) == node_value.one:
                output = inverted_output[node_value.d]
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = node_value.d
            else:
                output = node_value.zero
        elif input_list[0].get_value(output_node) == node_value.d_bar:
            if input_list[1].get_value(output_node) == node_value.d:
                output = node_value.one
            elif input_list[1].get_value(output_node) == node_value.one:
                output = inverted_output[node_value.d_bar]
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = node_value.d_bar
            else:
                output = node_value.zero

        for i in range(2, len(input_list)):
            if input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.one:
                output = inverted_output[output]
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.zero:
                    output = node_value.d
                elif output == node_value.one:
                    output = inverted_output[node_value.d]
                elif output == node_value.d:
                    output = node_value.zero
                elif output == node_value.d_bar:
                    output = node_value.one
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.zero:
                    output = node_value.d_bar
                elif output == node_value.one:
                    output = inverted_output[node_value.d_bar]
                elif output == node_value.d:
                    output = node_value.one
                elif output == node_value.d_bar:
                    output = node_value.zero
    
        return output
    
    @staticmethod
    def XNOR(input_list, output_node):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = None
        if input_list[0].get_value(output_node) == node_value.undefined:
            return node_value.undefined
        elif input_list[0].get_value(output_node) == node_value.one:
            output = input_list[1].get_value(output_node)
        elif input_list[0].get_value(output_node) == node_value.zero:
            output = inverted_output[input_list[1].get_value(output_node)]
        elif input_list[0].get_value(output_node) == node_value.d:
            if input_list[1].get_value(output_node) == node_value.d_bar:
                output = node_value.zero
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = inverted_output[node_value.d]
            elif input_list[1].get_value(output_node) == node_value.one:
                output = node_value.d
            else:
                output = node_value.one
        elif input_list[0].get_value(output_node) == node_value.d_bar:
            if input_list[1].get_value(output_node) == node_value.d:
                output = node_value.zero
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = inverted_output[node_value.d_bar]
            elif input_list[1].get_value(output_node) == node_value.one:
                output = node_value.d_bar
            else:
                output = node_value.one

        for i in range(2, len(input_list)):
            if input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.zero:
                output = inverted_output[output]
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.one:
                    output = node_value.d
                elif output == node_value.zero:
                    output = inverted_output[node_value.d]
                elif output == node_value.d:
                    output = node_value.one
                elif output == node_value.d_bar:
                    output = node_value.zero
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.one:
                    output = node_value.d_bar
                elif output == node_value.zero:
                    output = inverted_output[node_value.d_bar]
                elif output == node_value.d_bar:
                    output = node_value.one
                elif output == node_value.d:
                    output = node_value.zero

        return output
                
class fault_types:
    sa0 = 0
    sa1 = 1

In [165]:
#class to represent a fault
class fault:
    '''
    This class represents a fault.

    Attributes:
    fault_type: Type of fault, either stuck at 0 or stuck at 1
    fault_node: Node to which the fault is associated with
    fault_output: Output linked with the node where the fault is associated
    '''

    def __init__(self, fault_type, fault_node, fault_output = None):
        self.fault_type = fault_type
        self.fault_node = fault_node
        self.fault_output = fault_output
        self.fault_string = None
        if self.fault_output == None:
            if self.fault_type == fault_types.sa0:
                self.fault_string = f"{self.fault_node}-0"
            else:
                self.fault_string = f"{self.fault_node}-1"
        else:
            if self.fault_type == fault_types.sa0:
                self.fault_string = f"{self.fault_output}-{self.fault_node}-0"
            else:
                self.fault_string = f"{self.fault_output}-{self.fault_node}-1"

    def __repr__(self):
        '''
        Returns the string representation of a fault
        '''
        return self.fault_string

In [166]:
#class to represent a gate
class gate:
    '''
    This class is used to represent a gate

    Attributes:
    num_inputs: Number of inputs(int)
    type: Type of the gate(gate_type)
    input_nodes: Nodes that are fed in
    Output node: Output node of the gate
    '''

    def __init__(self, string_representation):
        '''
        This function creates a gate

        param[in] string_representation String representation of the gate
                                        The string after = in the circuit bench file
        '''
        self.input_nodes = []
        self.output_node = None
        self.type = None
        self.num_inputs = 0
        self.gate_string = ""
        self.gate_function = None

        self.__update_gate_type(string_representation)

    def __update_gate_type(self, string_representation: str):
        '''
        This function updates the gate type
        '''
        gate_dict = {"AND" : [gate_type.AND_gate, gate_type.AND],
                     "OR"  : [gate_type.OR_gate, gate_type.OR],
                     "NOT" : [gate_type.NOT_gate, gate_type.NOT],
                     "NOR" : [gate_type.NOR_gate, gate_type.NOR],
                     "NAND": [gate_type.NAND_gate, gate_type.NAND],
                     "XOR" : [gate_type.XOR_gate, gate_type.XOR],
                     "XNOR": [gate_type.XNOR_gate, gate_type.XNOR],
                     "BUFF": [gate_type.BUFF_gate, gate_type.BUFF]}

        self.gate_string = re.match(r'^[^()]+', string_representation)

        if self.gate_string:
            self.gate_string = self.gate_string.group()
        else:
            raise GateReprError(string_representation)
        

        if self.gate_string not in gate_dict:
            raise GateNotDefined(self.gate_string)
        else:
            self.type = gate_dict[self.gate_string][0]
            self.gate_function = gate_dict[self.gate_string][1]
            self.input_nodes = re.findall(r'\((.*?)\)', string_representation)[0]
            self.input_nodes = self.input_nodes.split(",")
            for i in range(len(self.input_nodes)):
                self.input_nodes[i] = self.input_nodes[i].lstrip()
            self.num_inputs = len(self.input_nodes)


    def get_output(self):
        self.output_node.value = self.gate_function(self.input_nodes, self.output_node)

    def __repr__(self):
        '''
        Return a string representing the gate information
        '''
        str_repr = f"{self.num_inputs}-input {self.gate_string} gate | Input nodes: "
        for node in self.input_nodes:
            str_repr = str_repr + f"{node},"
        str_repr = str_repr[:-1] + " | "
        str_repr = f"{str_repr}Output node: {self.output_node}"
        return str_repr

In [167]:
#class to represent a node
class node:
    '''
    This class represents a node in the circuit

    Attributes:
    name: Name of the node(string)
    type: Type of the node(node_type)
    value: Value of the node(node_value)
    gate_type: Type of gate(String)
    nodes_fed_in: List of nodes fed in(node)
    fault_list: List of faults associated with this node
    '''

    def __init__(self, circuit_bench_line: str):
        '''
        Initialize the object. 

        param[in] circuit_bench_line: A line in the circuit bench file
        '''
        self.name = None
        self.type = None
        self._value = node_value.undefined
        self.gate = None
        self.level = None
        self.fault_list = {fault_types.sa0: [], fault_types.sa1: []}
        self.selected_fault = None

        #check if it is an input node
        if ("INPUT" in circuit_bench_line.upper()):
            self.type = node_type.input_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
            self.level = 0
        elif ("OUTPUT" in circuit_bench_line.upper()):
            self.type = node_type.output_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
        else:
            #its an internal wire
            self.type = node_type.internal_wire
            self.name = circuit_bench_line.split("=")[0] #split at "=" symbol and the first element in the list
                                                         #is the node name
            self.name = self.name.rstrip() #remove any trailing white spaces
            self.__update_gate(circuit_bench_line)

    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, new_value):
        if (self.selected_fault == None) or \
            (isinstance(self.selected_fault, fault) and \
            (self.selected_fault.fault_output != None)):
            self._value = new_value
        else:
            if new_value in [node_value.d, node_value.d_bar]:
                #faulty value before fault site is not possible
                #hence raise an error
                raise FaultBeforeFaultsite(self.selected_fault, node_value.str_repr[new_value])
            else:
                if self.selected_fault.fault_type == fault_types.sa0:
                    if new_value == node_value.one:
                        new_value = node_value.d
                else:
                    if new_value == node_value.zero:
                        new_value = node_value.d_bar
                self._value = new_value

    def get_value(self, output_node):
        '''
        This function is used to get the value of the node based on the
        node fed out

        output_node: node fed out
        '''
        return_value = self._value
        if (self.selected_fault != None) and \
           (isinstance(self.selected_fault, fault) and \
            (((isinstance(self.selected_fault.fault_output, str)) and \
              (self.selected_fault.fault_output == "out")) or \
             ((isinstance(self.selected_fault.fault_output, node)) and \
              (self.selected_fault.fault_output.name == output_node.name)) or \
             ((self.selected_fault.fault_output == output_node)))):
            if self._value in [node_value.d, node_value.d_bar]:
                #faulty value before fault site is not possible
                #hence raise an error
                raise FaultBeforeFaultsite(self.selected_fault, node_value.str_repr[self._value])
            if self.selected_fault.fault_type == fault_types.sa0:
                #sa0 fault, return d if the node value is one
                if self._value == node_value.one:
                    return_value = node_value.d
            else:
                #sa1 fault, return d' if the node value is zero
                if self._value == node_value.zero:
                    return_value = node_value.d_bar

        return return_value

    
    def __update_gate(self, circuit_bench_line):
        '''
        Updates the gate information of a node
        '''
        gate_string_representation = circuit_bench_line.split("=")[1]
        gate_string_representation = gate_string_representation.lstrip()
        self.gate = gate(gate_string_representation)
        self.gate.output_node = self

    def update(self, circuit_bench_line):
        '''
        This function updates the nodes properties based on the new circuit bench file
        '''
        if "OUTPUT" in circuit_bench_line.upper():
            #node is an output node
            self.type = node_type.output_node
        else:
            #an assignment done to the node, update the gate type
            self.__update_gate(circuit_bench_line)

    def find_level(self):
        if self.level != None:
            #level already found
            return
        
        prev_level = 0

        for input_node in self.gate.input_nodes:
            if input_node.level == None:
                input_node.find_level()

            if input_node.level > prev_level:
                prev_level = input_node.level   

        self.level = prev_level + 1

    def create_fault_list(self, output_node = None):
        if output_node == None:
            if len(self.fault_list[fault_types.sa0]) == 0:
                self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self)) #creates a-0 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, "out"))
            if len(self.fault_list[fault_types.sa1]) == 0:
                self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self)) #creates a-1 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, "out"))
        else:
            self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, output_node))
            self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, output_node))
            return
        
        if self.gate != None:
            for input_node in self.gate.input_nodes:
                input_node.create_fault_list(self) #creates g-a-0 and g-a-1

    
    def select_fault(self, fault, output = None):
        '''
        Selects the fault based on the inputs given

        fault: Type of fault to select - sa0 or sa1
        Output: Output node associated with the fault
        '''

        if (isinstance(output, node) or (output == None)):
            for f in self.fault_list[fault]:
                if (isinstance(f.fault_output, node)):
                    if f.fault_output.name == output.name:
                        self.selected_fault = f
                        break
                elif ((f.fault_output == None)):
                    if f.fault_output == output:
                        self.selected_fault = f
                        break
        else:
            for f in self.fault_list[fault]:
                if (isinstance(f.fault_output, str)):
                    if f.fault_output == output:
                        self.selected_fault = f
                        break
        
    def __repr__(self):
        return self.name

In [201]:
#class to represent a circuit
class circuit:
    '''
    This class represents a circuit described in the circuit bench file

    Attributes:
    nodes: List of all the nodes(nodes)
    input_list: Index of input nodes in nodes list(int)
    output_list: Index of output nodes in nodes list(int)    
    '''
    
    def __init__(self, circuit_bench_file):
        '''
        Initializes the class

        param[in] circuit_bench_file: Path to the circuit bench file
        '''
        self.nodes = []
        self.input_list = []
        self.output_list = []
        self.internal_nodes = []
        self.node_index = {} #index of each node in the nodes list, key values for this dictionary are the node names
        self.levelized_nodes = None
        self.__fault_list_created = False
        self.num_levels = -1

        lines = self.__parse_circuit_bench_file(circuit_bench_file)
        #create nodes
        self.__create_nodes(lines)
        #check if all outputs are defined
        self.__check_output_definition()

    def __parse_circuit_bench_file(self, circuit_bench_file):
        '''
        This function parses the circuit bench file and sends out a list
        '''
        lines = []

        with open(circuit_bench_file, "r") as f:
            for line in f:
                line = line.replace("\n", "")
                if len(line) == 0:
                    continue
                lines.append(line)

        return lines
    
    def __create_nodes(self, lines):
        '''
        This function creates nodes from the lines read from the circuit bench file
        '''
        for line in lines:
            #check for comments
            if (line[0] == "#"):
                continue
            
            #check if node is already created - Possible when output is declared after using a wire
            node_name = re.findall(r'\((.*?)\)', line)[0]

            if "=" in line:
                node_name = line.split("=")[0]
                node_name = node_name.rstrip()

            if node_name in self.node_index:
                #update the node properties
                self.nodes[self.node_index[node_name]].update(line)
            else:
                #new node in the circuit
                #create a node and add it to the node list
                self.nodes.append(node(line))
                self.node_index[self.nodes[-1].name] = len(self.nodes) - 1
            

        #all the nodes are defined, now update the input nodes in the gates list to nodes
        #before this point, the input nodes in gate contains the node name alone and not the
        #node object
        for n in self.nodes:
            if n.gate != None:
                nodes_fed_in = n.gate.input_nodes
                for i in range(len(nodes_fed_in)):
                    nodes_fed_in[i] = self.nodes[self.node_index[nodes_fed_in[i]]]
                n.gate.input_nodes = nodes_fed_in

            #add nodes to the intern_node list
            if n.type == node_type.input_node:
                self.input_list.append(n)
            elif n.type == node_type.output_node:
                self.output_list.append(n)
            else:
                self.internal_nodes.append(n)

    
    def __check_output_definition(self):
        '''
        This function checks if all outputs are defined
        '''
        for node in self.nodes:
            if node.type == node_type.output_node:
                if node.gate == None:
                    raise FloatingOutput(node.name)
                
    def __repr__(self):
        '''
        This function returns the string representation of a circuit
        '''
        str_repr = "-------------------------------------------\n"
        str_repr = f"{str_repr}--------------Input Nodes------------------\n{self.input_list}\n"
        str_repr = f"{str_repr}-------------Output Nodes------------------\n{self.output_list}\n"
        str_repr = f"{str_repr}---------------Gate list-------------------\n"

        for node in self.nodes:
            if node.type != node_type.input_node:
                str_repr = f"{str_repr}{node.gate}\n"
        
        if self.levelized_nodes != None:
            str_repr = f"{str_repr}-------------Levelized circuit-------------\n"
            for level in self.levelized_nodes:
                str_repr = f"{str_repr}Level {level}: {self.levelized_nodes[level]}\n"

        return str_repr
    
    def levelize_circuit(self):
        '''
        This function levelizes the circuit and updates levelized_nodes
        '''
        if self.levelized_nodes != None:
            #circuit already levelized
            return
        
        self.levelized_nodes = {}
        
        for n in self.nodes:
            if n.level == None:
                n.find_level()

            if n.level in self.levelized_nodes:
                self.levelized_nodes[n.level].append(n)
            else:
                self.levelized_nodes[n.level] = [n]

        self.levelized_nodes = dict(sorted(self.levelized_nodes.items()))
        self.num_levels = len(self.levelized_nodes)

    def create_fault_list(self):
        '''
        This function creates the fault list for all the nodes in the circuit
        '''
        if self.__fault_list_created == True:
            return
        
        if self.levelized_nodes == None:
            raise CirNotLevelized()
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                n.create_fault_list()

        self.__fault_list_created = True
        
    def print_fault_list(self):
        '''
        This function prints the full fault list
        '''
        if not(self.__fault_list_created):
            print("Fault list not created")
            return
        
        total_faults = 0
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                print(f"Node: {n.name}, Faults: {n.fault_list}")
                total_faults += len(n.fault_list[0]) * 2

        print("Total number of faults: ", total_faults)

    def simulate(self, input_vector):
        '''
        This function simulates the circuit for the given input vector.
        The input vector should be a dictionary with the node names as the
        keys and values should be the node values
        '''

        for node_name in input_vector:
            self.nodes[self.node_index[node_name]].value = input_vector[node_name]
        
        for n in self.levelized_nodes[0]:
            if n.value == node_value.undefined:
                raise InputUndefined(n)
        
        for level in range(1, self.num_levels):
            for n in self.levelized_nodes[level]:
                n.gate.get_output()

    def select_fault(self, fault_string):
        fault_string_list = fault_string.split("-")
        input_node = None
        output_node = None
        fault_type = fault_types.sa0
        
        if len(fault_string_list) == 2:
            input_node = fault_string_list[0]
            fault_type = fault_string_list[1]
        else:
            input_node = fault_string_list[1]
            fault_type = fault_string_list[2] 
            output_node = fault_string_list[0]

        if input_node not in self.node_index:
            raise FaultReprError(fault_input= input_node)
        if fault_type not in ["0", "1"]:
            raise FaultReprError(fault_type = fault_type)
        if output_node != None:
            if output_node != "out":
                if output_node not in self.node_index:
                    raise FaultReprError(fault_output = output_node)
        
        if output_node != "out" and output_node != None:
            output_node = self.nodes[self.node_index[output_node]]
        
        if fault_type == "0":
            fault_type = fault_types.sa0
        else:
            fault_type = fault_types.sa1

        self.nodes[self.node_index[input_node]].select_fault(fault_type, output_node)

    def select_fault_from_user_input(self):
        print("Enter the fault to be selected in <output node name>-<input node name>-<0 or 1>  or <node name>-<0 or 1> format")
        print("To select the fault at the output, enter the fault format in \"out-<output node name>-<0 or 1>\"")
        fault_string = input("Enter fault string:")
        self.select_fault(fault_string)

    def __print_node_values_as_table(self, input_list, entries_per_row):
        '''
        prints node values as tables
        '''
        output_repr = {node_value.zero     : "0",
                       node_value.one      : "1",
                       node_value.d        : "D",
                       node_value.d_bar    : "D'",
                       node_value.undefined: "X"}
        
        index_row_1 = 0
        index_row_2 = 0
        for i in range(entries_per_row, len(input_list), entries_per_row):
            print("Node  ", end = "")
            while index_row_1 < i:
                print("| {:^5} ".format(input_list[index_row_1].name), end = "")
                index_row_1 += 1
            print("|")
            print("Value ", end = "")
            while index_row_2 < i:
                print("| {:^5} ".format(output_repr[input_list[index_row_2].value]), end = "")
                index_row_2 += 1
            print("|\n")


        if index_row_1 < len(input_list):
            print("Node  ", end = "")
            while index_row_1 < len(input_list):
                print("| {:^5} ".format(input_list[index_row_1].name), end = "")
                index_row_1 += 1
            print("|")
            print("Value ", end = "")
            while index_row_2 < len(input_list):
                print("| {:^5} ".format(output_repr[input_list[index_row_2].value]), end = "")
                index_row_2 += 1
            print("|")
        

    def print_node_values(self, input_entries_per_row = 5, internal_entries_per_row = 5, output_entries_per_row = 5):
        '''
        This function prints the values of all the nodes in the circuit. Output
        will be printed in the ascending order of levels 
        '''
        if (input_entries_per_row > 0):
            print(f'------Input Nodes------')
            self.__print_node_values_as_table(self.input_list, input_entries_per_row)

        if (internal_entries_per_row > 0):
            print(f'\n------Internal Nodes------')
            self.__print_node_values_as_table(self.internal_nodes, internal_entries_per_row)

        if (output_entries_per_row > 0):
            print(f'\n------Output Nodes------')
            self.__print_node_values_as_table(self.output_list, output_entries_per_row)


    def reset_circuit(self):
        '''
        Resets all the circuit parameters
        '''
        for n in self.input_list:
            n.value = node_value.undefined
            n.selected_fault = None

        for n in self.internal_nodes:
            n.value = node_value.undefined
            n.selected_fault = None

        for n in self.output_list:
            n.value = node_value.undefined
            n.selected_fault = None
    
    

In [224]:
def create_input_vector(num_inputs, choices = [node_value.zero, node_value.one]):
    
    total_input_vector = len(choices) ** num_inputs
    input_vectors = []
    for i in range(total_input_vector):
        for j in range(num_inputs):
            divisor = 1
            if j != 0:
                divisor = len(choices) ** j 
            value = [choices[(i // divisor) % len(choices)]]
            if j == 0:
                input_vectors.append(value)
            else:
                input_vectors[-1] = input_vectors[-1] + value

    return input_vectors

def print_tables(dict_list):
    '''
    This functions prints the list of dictionary in a table format
    '''
    headers = dict_list[0].keys()
    for h in headers:
        print("| {:<5} ".format(h), end = "")
    print("|")

    for d in dict_list:
        for h in d:
            print("| {:<5} ".format(node_value.str_repr[d[h]]), end = "")
        print("|")


def test_circuit(circuit_under_test: circuit, iv_list, print_result = True):
    '''
    This function tests the given circuit with the input vector in the iv_list
    '''
    input_vector = {}
    
    for n in circuit_under_test.input_list:
        input_vector[n.name] = node_value.undefined

    results = []
    
    for iv in iv_list:
        i = 0
        for n in input_vector:
            input_vector[n] = iv[i]
            i = i + 1
        circuit_under_test.simulate(input_vector)
        results.append(copy.deepcopy(input_vector))
        for n in circuit_under_test.output_list:
            results[-1][n.name] = n.value
    
    if (print_result):
        print_tables(results)

def test_fault_simulation(circuit_under_test: circuit, input_vector, fault_string):
    '''
    This function activates the specified fault and check if the fault can be detected
    '''

    circuit_under_test.create_fault_list()
    circuit_under_test.select_fault(fault_string)
    test_circuit(circuit_under_test, [input_vector], print_result = False)

    fault_detected = False
    for n in circuit_under_test.output_list:
        if n.value in [node_value.d, node_value.d_bar]:
            fault_detected = True
    
    return fault_detected

def test_full_fault_list_detection(circuit_under_test: circuit, input_vector):
    '''
    This function tests if all the faults in the circuit are detected by the test vector
    '''
    circuit_under_test.create_fault_list()
    results = {}
    num_faults_detected = 0

    for n in circuit_under_test.nodes:
        for f_type in [fault_types.sa0, fault_types.sa1]:
            for f in n.fault_list[f_type]:
                fault_detected = test_fault_simulation(circuit_under_test, input_vector, f.fault_string)
                if fault_detected:
                    num_faults_detected += 1
                    results[f.fault_string] = "Y"
                else:
                    results[f.fault_string] = "N"
                circuit_under_test.reset_circuit()

    index_row_1 = 0
    index_row_2 = 0
    entries_per_row = 15
    input_list = list(results.keys())
    print("Total number of faults: ", len(input_list))
    print("Total number of faults detected: ", num_faults_detected)
    print("Percentage of faults detected: {}%".format((num_faults_detected / len(results)) * 100))
    for i in range(entries_per_row, len(input_list), entries_per_row):
        print("Fault     ", end = "")
        while index_row_1 < i:
            print("| {:^10} ".format(input_list[index_row_1]), end = "")
            index_row_1 += 1
        print("|")
        print("Detected? ", end = "")
        while index_row_2 < i:
            print("| {:^10} ".format(results[input_list[index_row_2]]), end = "")
            index_row_2 += 1
        print("|\n")


    if index_row_1 < len(input_list):
        print("Fault     ", end = "")
        while index_row_1 < len(input_list):
            print("| {:^10} ".format(input_list[index_row_1]), end = "")
            index_row_1 += 1
        print("|")
        print("Detected? ", end = "")
        while index_row_2 < len(input_list):
            print("| {:^10} ".format(results[input_list[index_row_2]]), end = "")
            index_row_2 += 1
        print("|")
    
    

## Verifying basic functionalities
1. Fault simulation
2. Circuit simulation without inputs
3. Internal nodes classification

In [170]:
circuit_under_test = circuit("p1.txt")
circuit_under_test.levelize_circuit()
print(circuit_under_test)
circuit_under_test.create_fault_list()
print("--------Full fault list testing--------")
total_faults = 0
for level in circuit_under_test.levelized_nodes:
    for n in circuit_under_test.levelized_nodes[level]:
        print(n.fault_list)
        total_faults += len(n.fault_list[0]) * 2

print("Total number of faults: ", total_faults)

print("--------Circuit simulation testing without input assignment(undefined)--------")
circuit_under_test.print_node_values()

print("---------Verifying internal nodes list---------")
print(circuit_under_test.internal_nodes)

-------------------------------------------
--------------Input Nodes------------------
[a, b, c]
-------------Output Nodes------------------
[w, x, y, z]
---------------Gate list-------------------
3-input AND gate | Input nodes: a,b,c | Output node: w
3-input OR gate | Input nodes: d,e,f | Output node: x
2-input OR gate | Input nodes: g,h | Output node: y
1-input NOT gate | Input nodes: c | Output node: z
1-input NOT gate | Input nodes: a | Output node: a'
1-input NOT gate | Input nodes: b | Output node: b'
1-input NOT gate | Input nodes: c | Output node: c'
2-input AND gate | Input nodes: a,b' | Output node: d
3-input AND gate | Input nodes: a',b,c | Output node: e
3-input AND gate | Input nodes: a,b,c' | Output node: f
2-input AND gate | Input nodes: b',c | Output node: g
2-input AND gate | Input nodes: b,c' | Output node: h
-------------Levelized circuit-------------
Level 0: [a, b, c]
Level 1: [w, z, a', b', c']
Level 2: [d, e, f, g, h]
Level 3: [x, y]

--------Full fault list te

### Test to Verify circuit simulation with 1 and 0

All gate functionalities are verified

In [171]:
circuit_under_test = circuit("test1.txt")
circuit_under_test.levelize_circuit()
iv_list = create_input_vector(len(circuit_under_test.input_list))
test_circuit(circuit_under_test, iv_list)

| a     | b     | and   | or    | nand  | nor   | xor   | xnor  | not   |
| 0     | 0     | 0     | 0     | 1     | 1     | 0     | 1     | 1     |
| 1     | 0     | 0     | 1     | 1     | 0     | 1     | 0     | 0     |
| 0     | 1     | 0     | 1     | 1     | 0     | 1     | 0     | 1     |
| 1     | 1     | 1     | 1     | 0     | 0     | 0     | 1     | 0     |


### Test to verify basic circuit simulation with 0, 1, D, D'
All gates are verified

In [172]:
choices = [node_value.zero, node_value.one, node_value.d, node_value.d_bar]
circuit_under_test = circuit("test1.txt")
circuit_under_test.levelize_circuit()
iv_list = create_input_vector(len(circuit_under_test.input_list), choices)
test_circuit(circuit_under_test, iv_list)

| a     | b     | and   | or    | nand  | nor   | xor   | xnor  | not   |
| 0     | 0     | 0     | 0     | 1     | 1     | 0     | 1     | 1     |
| 1     | 0     | 0     | 1     | 1     | 0     | 1     | 0     | 0     |
| D     | 0     | 0     | D     | 1     | D'    | D     | D'    | D'    |
| D'    | 0     | 0     | D'    | 1     | D     | D'    | D     | D     |
| 0     | 1     | 0     | 1     | 1     | 0     | 1     | 0     | 1     |
| 1     | 1     | 1     | 1     | 0     | 0     | 0     | 1     | 0     |
| D     | 1     | D     | 1     | D'    | 0     | D'    | D     | D'    |
| D'    | 1     | D'    | 1     | D     | 0     | D     | D'    | D     |
| 0     | D     | 0     | D     | 1     | D'    | D     | D'    | 1     |
| 1     | D     | D     | 1     | D'    | 0     | D'    | D     | 0     |
| D     | D     | D     | D     | D'    | D'    | 0     | 1     | D'    |
| D'    | D     | 0     | 1     | 1     | 0     | 1     | 0     | D     |
| 0     | D'    | 0     | D'    | 1   

In [173]:
circuit_under_test = circuit("test2.txt")
circuit_under_test.levelize_circuit()
iv_list = create_input_vector(len(circuit_under_test.input_list))
test_circuit(circuit_under_test, iv_list)

| a     | b     | c     | d     | k     | l     | m     |
| 0     | 0     | 0     | 0     | 1     | 1     | 1     |
| 1     | 0     | 0     | 0     | 1     | 1     | 1     |
| 0     | 1     | 0     | 0     | 1     | 1     | 1     |
| 1     | 1     | 0     | 0     | 1     | 1     | 1     |
| 0     | 0     | 1     | 0     | 1     | 1     | 1     |
| 1     | 0     | 1     | 0     | 1     | 1     | 1     |
| 0     | 1     | 1     | 0     | 1     | 1     | 1     |
| 1     | 1     | 1     | 0     | 1     | 1     | 1     |
| 0     | 0     | 0     | 1     | 1     | 1     | 1     |
| 1     | 0     | 0     | 1     | 1     | 1     | 1     |
| 0     | 1     | 0     | 1     | 1     | 1     | 1     |
| 1     | 1     | 0     | 1     | 1     | 1     | 1     |
| 0     | 0     | 1     | 1     | 1     | 1     | 1     |
| 1     | 0     | 1     | 1     | 1     | 1     | 1     |
| 0     | 1     | 1     | 1     | 1     | 1     | 1     |
| 1     | 1     | 1     | 1     | 1     | 1     | 1     |


In [174]:
choices = [node_value.zero, node_value.one, node_value.d, node_value.d_bar]
circuit_under_test = circuit("test2.txt")
circuit_under_test.levelize_circuit()
iv_list = create_input_vector(len(circuit_under_test.input_list), choices)
test_circuit(circuit_under_test, iv_list)

| a     | b     | c     | d     | k     | l     | m     |
| 0     | 0     | 0     | 0     | 1     | 1     | 1     |
| 1     | 0     | 0     | 0     | 1     | 1     | 1     |
| D     | 0     | 0     | 0     | 1     | 1     | 1     |
| D'    | 0     | 0     | 0     | 1     | 1     | 1     |
| 0     | 1     | 0     | 0     | 1     | 1     | 1     |
| 1     | 1     | 0     | 0     | 1     | 1     | 1     |
| D     | 1     | 0     | 0     | 1     | 1     | 1     |
| D'    | 1     | 0     | 0     | 1     | 1     | 1     |
| 0     | D     | 0     | 0     | 1     | 1     | 1     |
| 1     | D     | 0     | 0     | 1     | 1     | 1     |
| D     | D     | 0     | 0     | 1     | 1     | 1     |
| D'    | D     | 0     | 0     | 1     | 1     | 1     |
| 0     | D'    | 0     | 0     | 1     | 1     | 1     |
| 1     | D'    | 0     | 0     | 1     | 1     | 1     |
| D     | D'    | 0     | 0     | 1     | 1     | 1     |
| D'    | D'    | 0     | 0     | 1     | 1     | 1     |
| 0     | 0   

### Test to verify fault simulation
test3.txt bench file is used to verify the fault simulation results. Circuit is from 9/12 class

In [175]:
circuit_under_test = circuit("test3.txt")
circuit_under_test.levelize_circuit()
circuit_under_test.create_fault_list()
circuit_under_test.select_fault("m-0")
test_circuit(circuit_under_test, [[1, 1, 0, 0, 0, 0]], False)
print(circuit_under_test)
circuit_under_test.print_node_values()

-------------------------------------------
--------------Input Nodes------------------
[A, B, C, D, E, F]
-------------Output Nodes------------------
[S, T]
---------------Gate list-------------------
2-input NOR gate | Input nodes: p,g | Output node: S
3-input AND gate | Input nodes: p,o,k | Output node: T
2-input AND gate | Input nodes: A,B | Output node: g
1-input NOT gate | Input nodes: C | Output node: n
1-input NOT gate | Input nodes: D | Output node: o
2-input NOR gate | Input nodes: E,F | Output node: k
2-input OR gate | Input nodes: g,C | Output node: m
2-input NAND gate | Input nodes: m,n | Output node: p
-------------Levelized circuit-------------
Level 0: [A, B, C, D, E, F]
Level 1: [g, n, o, k]
Level 2: [m]
Level 3: [p]
Level 4: [S, T]

------Input Nodes------
Node  |   A   |   B   |   C   |   D   |   E   |
Value |   1   |   1   |   0   |   0   |   0   |

Node  |   F   |
Value |   0   |

------Internal Nodes------
Node  |   g   |   n   |   o   |   k   |   m   |
Value |   

In [176]:
test_circuit(circuit_under_test, [[1, 1, 0, 0, 0, 0]], False)
circuit_under_test.print_node_values()

------Input Nodes------
Node  |   A   |   B   |   C   |   D   |   E   |
Value |   1   |   1   |   0   |   0   |   0   |

Node  |   F   |
Value |   0   |

------Internal Nodes------
Node  |   g   |   n   |   o   |   k   |   m   |
Value |   1   |   1   |   1   |   1   |   D   |

Node  |   p   |
Value |  D'   |

------Output Nodes------
Node  |   S   |   T   |
Value |   0   |  D'   |


In [177]:
circuit_under_test.select_fault("m-1")
test_circuit(circuit_under_test, [[0, 0, 0, 1, 0, 0]], False)
circuit_under_test.print_node_values()

------Input Nodes------
Node  |   A   |   B   |   C   |   D   |   E   |
Value |   0   |   0   |   0   |   1   |   0   |

Node  |   F   |
Value |   0   |

------Internal Nodes------
Node  |   g   |   n   |   o   |   k   |   m   |
Value |   0   |   1   |   0   |   1   |  D'   |

Node  |   p   |
Value |   D   |

------Output Nodes------
Node  |   S   |   T   |
Value |  D'   |   0   |


### 2.A.i Fault listing for hw1.bench

In [178]:
circuit_hw1 = circuit("bench_files/hw1.bench")
circuit_hw1.levelize_circuit()
circuit_hw1.create_fault_list()
circuit_hw1.print_fault_list()

Node: a, Faults: {0: [a-0, w-a-0, a'-a-0, d-a-0, f-a-0], 1: [a-1, w-a-1, a'-a-1, d-a-1, f-a-1]}
Node: b, Faults: {0: [b-0, w-b-0, b'-b-0, e-b-0, f-b-0, h-b-0], 1: [b-1, w-b-1, b'-b-1, e-b-1, f-b-1, h-b-1]}
Node: c, Faults: {0: [c-0, c'-c-0, w-c-0, z-c-0, e-c-0, g-c-0], 1: [c-1, c'-c-1, w-c-1, z-c-1, e-c-1, g-c-1]}
Node: c', Faults: {0: [c'-0, f-c'-0, h-c'-0], 1: [c'-1, f-c'-1, h-c'-1]}
Node: w, Faults: {0: [w-0, out-w-0], 1: [w-1, out-w-1]}
Node: z, Faults: {0: [z-0, out-z-0], 1: [z-1, out-z-1]}
Node: a', Faults: {0: [a'-0, e-a'-0], 1: [a'-1, e-a'-1]}
Node: b', Faults: {0: [b'-0, d-b'-0, g-b'-0], 1: [b'-1, d-b'-1, g-b'-1]}
Node: e, Faults: {0: [e-0, x-e-0], 1: [e-1, x-e-1]}
Node: d, Faults: {0: [d-0, x-d-0], 1: [d-1, x-d-1]}
Node: f, Faults: {0: [f-0, x-f-0], 1: [f-1, x-f-1]}
Node: g, Faults: {0: [g-0, y-g-0], 1: [g-1, y-g-1]}
Node: h, Faults: {0: [h-0, y-h-0], 1: [h-1, y-h-1]}
Node: x, Faults: {0: [x-0, out-x-0], 1: [x-1, out-x-1]}
Node: y, Faults: {0: [y-0, out-y-0], 1: [y-1, out-y-1

#### 2.A.ii Fault listing for c432.bench

In [179]:
circuit_hw1 = circuit("bench_files/c432.bench")
circuit_hw1.levelize_circuit()
circuit_hw1.create_fault_list()
circuit_hw1.print_fault_list()

Node: 1, Faults: {0: [1-0, 118-1-0, 242-1-0], 1: [1-1, 118-1-1, 242-1-1]}
Node: 4, Faults: {0: [4-0, 119-4-0, 154-4-0, 380-4-0], 1: [4-1, 119-4-1, 154-4-1, 380-4-1]}
Node: 8, Faults: {0: [8-0, 157-8-0, 334-8-0], 1: [8-1, 157-8-1, 334-8-1]}
Node: 11, Faults: {0: [11-0, 122-11-0, 246-11-0], 1: [11-1, 122-11-1, 246-11-1]}
Node: 14, Faults: {0: [14-0, 158-14-0, 371-14-0], 1: [14-1, 158-14-1, 371-14-1]}
Node: 17, Faults: {0: [17-0, 123-17-0, 159-17-0, 381-17-0], 1: [17-1, 123-17-1, 159-17-1, 381-17-1]}
Node: 21, Faults: {0: [21-0, 183-21-0, 336-21-0], 1: [21-1, 183-21-1, 336-21-1]}
Node: 24, Faults: {0: [24-0, 126-24-0, 250-24-0], 1: [24-1, 126-24-1, 250-24-1]}
Node: 27, Faults: {0: [27-0, 184-27-0, 372-27-0], 1: [27-1, 184-27-1, 372-27-1]}
Node: 30, Faults: {0: [30-0, 127-30-0, 162-30-0, 386-30-0], 1: [30-1, 127-30-1, 162-30-1, 386-30-1]}
Node: 34, Faults: {0: [34-0, 185-34-0, 338-34-0], 1: [34-1, 185-34-1, 338-34-1]}
Node: 37, Faults: {0: [37-0, 130-37-0, 254-37-0], 1: [37-1, 130-37-1, 25

#### 2.B.i Circuit simulation with all 0's for hw1.bench

In [180]:
circuit_hw1 = circuit("bench_files/hw1.bench")
circuit_hw1.levelize_circuit()
iv = [[0 for i in range(len(circuit_hw1.input_list))]]
test_circuit(circuit_hw1, iv, print_result= False)
circuit_hw1.print_node_values()

------Input Nodes------
Node  |   a   |   b   |   c   |
Value |   0   |   0   |   0   |

------Internal Nodes------
Node  |  c'   |   e   |   d   |  a'   |  b'   |
Value |   1   |   0   |   0   |   1   |   1   |

Node  |   f   |   g   |   h   |
Value |   0   |   0   |   0   |

------Output Nodes------
Node  |   w   |   x   |   y   |   z   |
Value |   0   |   0   |   0   |   1   |


#### 2.B.i Circuit simulation with all 1's for hw1.bench

In [181]:
circuit_hw1 = circuit("bench_files/hw1.bench")
circuit_hw1.levelize_circuit()
iv = [[1 for i in range(len(circuit_hw1.input_list))]]
test_circuit(circuit_hw1, iv, print_result= False)
circuit_hw1.print_node_values()

------Input Nodes------
Node  |   a   |   b   |   c   |
Value |   1   |   1   |   1   |

------Internal Nodes------
Node  |  c'   |   e   |   d   |  a'   |  b'   |
Value |   0   |   0   |   0   |   0   |   0   |

Node  |   f   |   g   |   h   |
Value |   0   |   0   |   0   |

------Output Nodes------
Node  |   w   |   x   |   y   |   z   |
Value |   1   |   0   |   0   |   0   |


#### 2.B.ii Circuit simulation with all 0's for c880.bench

In [182]:
circuit_c880 = circuit("bench_files/c880.bench")
circuit_c880.levelize_circuit()
iv = [[0 for i in range(len(circuit_c880.input_list))]]
test_circuit(circuit_c880, iv, print_result= False)
circuit_c880.print_node_values(15, 0, 15)

------Input Nodes------
Node  |   1   |   8   |  13   |  17   |  26   |  29   |  36   |  42   |  51   |  55   |  59   |  68   |  72   |  73   |  74   |
Value |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |

Node  |  75   |  80   |  85   |  86   |  87   |  88   |  89   |  90   |  91   |  96   |  101  |  106  |  111  |  116  |  121  |
Value |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |

Node  |  126  |  130  |  135  |  138  |  143  |  146  |  149  |  152  |  153  |  156  |  159  |  165  |  171  |  177  |  183  |
Value |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |

Node  |  189  |  195  |  201  |  207  |  210  |  219  |  228  |  237  |  246  |  255  |  259  |  260  |  261  |  267  |  268  |
Value |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0  

#### 2.B.ii Circuit simulation with all 1's for c880.bench

In [183]:
circuit_c880 = circuit("bench_files/c880.bench")
circuit_c880.levelize_circuit()
iv = [[1 for i in range(len(circuit_c880.input_list))]]
test_circuit(circuit_c880, iv, print_result= False)
circuit_c880.print_node_values(15, 0, 15)

------Input Nodes------
Node  |   1   |   8   |  13   |  17   |  26   |  29   |  36   |  42   |  51   |  55   |  59   |  68   |  72   |  73   |  74   |
Value |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |

Node  |  75   |  80   |  85   |  86   |  87   |  88   |  89   |  90   |  91   |  96   |  101  |  106  |  111  |  116  |  121  |
Value |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |

Node  |  126  |  130  |  135  |  138  |  143  |  146  |  149  |  152  |  153  |  156  |  159  |  165  |  171  |  177  |  183  |
Value |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |

Node  |  189  |  195  |  201  |  207  |  210  |  219  |  228  |  237  |  246  |  255  |  259  |  260  |  261  |  267  |  268  |
Value |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1   |   1  

#### 2.C.1.a Fault 22-10-0 simulation with all 0 input vector on c17.bench

In [184]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [0 for i in range(len(circuit_c17.input_list))]  
fault_detected = test_fault_simulation(circuit_c17, iv, "22-10-0")
fault_string = "22-10-0 fault not detected!!!!"
if fault_detected:
    fault_string = "22-10-0 fault detected!!!!"
print(fault_string)
circuit_c17.print_node_values(10, 10, 10)

22-10-0 fault detected!!!!
------Input Nodes------
Node  |   1   |   2   |   3   |   6   |   7   |
Value |   0   |   0   |   0   |   0   |   0   |

------Internal Nodes------
Node  |  10   |  11   |  16   |  19   |
Value |   1   |   1   |   1   |   1   |

------Output Nodes------
Node  |  22   |  23   |
Value |  D'   |   0   |


#### 2.C.1.b Fault 22-10-0 simulation with all 1 input vector on c17.bench

In [185]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [1 for i in range(len(circuit_c17.input_list))]  
fault_detected = test_fault_simulation(circuit_c17, iv, "22-10-0")
fault_string = "22-10-0 fault not detected!!!!"
if fault_detected:
    fault_string = "22-10-0 fault detected!!!!"
print(fault_string)
circuit_c17.print_node_values(10, 10, 10)

22-10-0 fault not detected!!!!
------Input Nodes------
Node  |   1   |   2   |   3   |   6   |   7   |
Value |   1   |   1   |   1   |   1   |   1   |

------Internal Nodes------
Node  |  10   |  11   |  16   |  19   |
Value |   0   |   0   |   1   |   1   |

------Output Nodes------
Node  |  22   |  23   |
Value |   1   |   0   |


#### 2.C.1.c Fault 22-10-1 simulation with all 0 input vector on c17.bench

In [186]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [0 for i in range(len(circuit_c17.input_list))]  
fault_detected = test_fault_simulation(circuit_c17, iv, "22-10-1")
fault_string = "22-10-1 fault not detected!!!!"
if fault_detected:
    fault_string = "22-10-1 fault detected!!!!"
print(fault_string)
circuit_c17.print_node_values(10, 10, 10)

22-10-1 fault not detected!!!!
------Input Nodes------
Node  |   1   |   2   |   3   |   6   |   7   |
Value |   0   |   0   |   0   |   0   |   0   |

------Internal Nodes------
Node  |  10   |  11   |  16   |  19   |
Value |   1   |   1   |   1   |   1   |

------Output Nodes------
Node  |  22   |  23   |
Value |   0   |   0   |


#### 2.C.1.d Fault 22-10-1 simulation with all 1 input vector on c17.bench

In [187]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [1 for i in range(len(circuit_c17.input_list))]  
fault_detected = test_fault_simulation(circuit_c17, iv, "22-10-1")
fault_string = "22-10-1 fault not detected!!!!"
if fault_detected:
    fault_string = "22-10-1 fault detected!!!!"
print(fault_string)
circuit_c17.print_node_values(10, 10, 10)

22-10-1 fault detected!!!!
------Input Nodes------
Node  |   1   |   2   |   3   |   6   |   7   |
Value |   1   |   1   |   1   |   1   |   1   |

------Internal Nodes------
Node  |  10   |  11   |  16   |  19   |
Value |   0   |   0   |   1   |   1   |

------Output Nodes------
Node  |  22   |  23   |
Value |   D   |   0   |


#### 2.C.2.a w-0 fault testing for hw1.bench with TV=111

In [194]:
circuit_hw1 = circuit("bench_files/hw1.bench")
circuit_hw1.levelize_circuit()
iv = [1 for i in range(len(circuit_hw1.input_list))]
fault_string = "w-0"
fault_detected = test_fault_simulation(circuit_hw1, iv, fault_string)
if fault_detected:
    fault_string = f"{fault_string} fault detected!!!!"
else:
    fault_string = f"{fault_string} fault not detected!!!!"
print(fault_string)
circuit_hw1.print_node_values(10, 10, 10)

w-0 fault detected!!!!
------Input Nodes------
Node  |   a   |   b   |   c   |
Value |   1   |   1   |   1   |

------Internal Nodes------
Node  |  c'   |   e   |   d   |  a'   |  b'   |   f   |   g   |   h   |
Value |   0   |   0   |   0   |   0   |   0   |   0   |   0   |   0   |

------Output Nodes------
Node  |   w   |   x   |   y   |   z   |
Value |   D   |   0   |   0   |   0   |


#### 2.C.2.b w-0 fault testing for hw1.bench with TV=110 (fault is not detectable with the TV)

In [195]:
circuit_hw1 = circuit("bench_files/hw1.bench")
circuit_hw1.levelize_circuit()
iv = [1, 1, 0]
fault_string = "w-0"
fault_detected = test_fault_simulation(circuit_hw1, iv, fault_string)
if fault_detected:
    fault_string = f"{fault_string} fault detected!!!!"
else:
    fault_string = f"{fault_string} fault not detected!!!!"
print(fault_string)
circuit_hw1.print_node_values(10, 10, 10)

w-0 fault not detected!!!!
------Input Nodes------
Node  |   a   |   b   |   c   |
Value |   1   |   1   |   0   |

------Internal Nodes------
Node  |  c'   |   e   |   d   |  a'   |  b'   |   f   |   g   |   h   |
Value |   1   |   0   |   0   |   0   |   0   |   1   |   0   |   1   |

------Output Nodes------
Node  |   w   |   x   |   y   |   z   |
Value |   0   |   1   |   1   |   1   |


#### 2.C.3.a Full fault testing for c17.bench with all 0 test vectors

In [225]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [0 for i in range(len(circuit_c17.input_list))]  
test_full_fault_list_detection(circuit_c17, iv)

Total number of faults:  50
Total number of faults detected:  13
Percentage of faults detected: 26.0%
Fault     |    1-0     |   10-1-0   |    1-1     |   10-1-1   |    2-0     |   16-2-0   |    2-1     |   16-2-1   |    3-0     |   10-3-0   |   11-3-0   |    3-1     |   10-3-1   |   11-3-1   |    6-0     |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     Y      |     Y      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |   11-6-0   |    6-1     |   11-6-1   |    7-0     |   19-7-0   |    7-1     |   19-7-1   |    22-0    |  out-22-0  |    22-1    |  out-22-1  |    23-0    |  out-23-0  |    23-1    |  out-23-1  |
Detected? |     N      |     N      |     N      |     N      |     N      |     Y      |     Y      |     N      |     N      |     Y      |     N      |     N      |     N      |     Y      |     N      |

Fault     |    10-0    |  22-10-0   |    10-1    |  22-10-1   |    1

#### 2.C.3.a Full fault testing for c17.bench with all 1 test vectors

In [226]:
circuit_c17 = circuit("bench_files/c17.bench")
circuit_c17.levelize_circuit()
iv = [1 for i in range(len(circuit_c17.input_list))]  
test_full_fault_list_detection(circuit_c17, iv)

Total number of faults:  50
Total number of faults detected:  18
Percentage of faults detected: 36.0%
Fault     |    1-0     |   10-1-0   |    1-1     |   10-1-1   |    2-0     |   16-2-0   |    2-1     |   16-2-1   |    3-0     |   10-3-0   |   11-3-0   |    3-1     |   10-3-1   |   11-3-1   |    6-0     |
Detected? |     Y      |     Y      |     N      |     N      |     N      |     N      |     N      |     N      |     Y      |     Y      |     Y      |     N      |     N      |     N      |     Y      |

Fault     |   11-6-0   |    6-1     |   11-6-1   |    7-0     |   19-7-0   |    7-1     |   19-7-1   |    22-0    |  out-22-0  |    22-1    |  out-22-1  |    23-0    |  out-23-0  |    23-1    |  out-23-1  |
Detected? |     Y      |     N      |     N      |     N      |     N      |     N      |     N      |     Y      |     N      |     N      |     N      |     N      |     N      |     Y      |     N      |

Fault     |    10-0    |  22-10-0   |    10-1    |  22-10-1   |    1

#### 2.C.4.a Full fault testing for c880.bench with all 0 test vectors


In [227]:
circuit_c880 = circuit("bench_files/c880.bench")
circuit_c880.levelize_circuit()
iv = [0 for i in range(len(circuit_c880.input_list))]  
test_full_fault_list_detection(circuit_c880, iv)

Total number of faults:  2396
Total number of faults detected:  397
Percentage of faults detected: 16.569282136894824%
Fault     |    1-0     |  269-1-0   |  270-1-0   |  276-1-0   |  279-1-0   |  280-1-0   |  483-1-0   |    1-1     |  269-1-1   |  270-1-1   |  276-1-1   |  279-1-1   |  280-1-1   |  483-1-1   |    8-0     |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  269-8-0   |  279-8-0   |  280-8-0   |  309-8-0   |    8-1     |  269-8-1   |  279-8-1   |  280-8-1   |  309-8-1   |    13-0    |  269-13-0  |  270-13-0  |  280-13-0  |    13-1    |  269-13-1  |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  270-13-1  |  280-13-1  |    17-0    | 

#### 2.C.4.b Full fault testing for c880.bench with all 1 test vectors

In [228]:
circuit_c880 = circuit("bench_files/c880.bench")
circuit_c880.levelize_circuit()
iv = [0 for i in range(len(circuit_c880.input_list))]  
test_full_fault_list_detection(circuit_c880, iv)

Total number of faults:  2396
Total number of faults detected:  397
Percentage of faults detected: 16.569282136894824%
Fault     |    1-0     |  269-1-0   |  270-1-0   |  276-1-0   |  279-1-0   |  280-1-0   |  483-1-0   |    1-1     |  269-1-1   |  270-1-1   |  276-1-1   |  279-1-1   |  280-1-1   |  483-1-1   |    8-0     |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  269-8-0   |  279-8-0   |  280-8-0   |  309-8-0   |    8-1     |  269-8-1   |  279-8-1   |  280-8-1   |  309-8-1   |    13-0    |  269-13-0  |  270-13-0  |  280-13-0  |    13-1    |  269-13-1  |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  270-13-1  |  280-13-1  |    17-0    | 

#### 2.C.4.c Full fault testing for c880.bench with random test vectors

In [231]:
import random
circuit_c880 = circuit("bench_files/c880.bench")
circuit_c880.levelize_circuit()
iv = [random.choice([0, 1]) for i in range(len(circuit_c880.input_list))]
test_full_fault_list_detection(circuit_c880, iv)

Total number of faults:  2396
Total number of faults detected:  601
Percentage of faults detected: 25.08347245409015%
Fault     |    1-0     |  269-1-0   |  270-1-0   |  276-1-0   |  279-1-0   |  280-1-0   |  483-1-0   |    1-1     |  269-1-1   |  270-1-1   |  276-1-1   |  279-1-1   |  280-1-1   |  483-1-1   |    8-0     |
Detected? |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  269-8-0   |  279-8-0   |  280-8-0   |  309-8-0   |    8-1     |  269-8-1   |  279-8-1   |  280-8-1   |  309-8-1   |    13-0    |  269-13-0  |  270-13-0  |  280-13-0  |    13-1    |  269-13-1  |
Detected? |     N      |     N      |     N      |     N      |     Y      |     Y      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |     N      |

Fault     |  270-13-1  |  280-13-1  |    17-0    |  